In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("/home/Brandon/Documents/feedback_helper_functions/")
from transfer import *
import filters as fl

In [3]:
a, b = fl.find_chebyshev_ab(4, .9)
n = 4
print(f"a = {a}, b = {b}")
offset_start = 180/(2*n)
for i in range(n):
    a, b = fl.find_chebyshev_ab(i, .9)
    cur_frac = frac([1], [1, a*np.cos(np.deg2rad(offset_start + i*(180/n))) + j*b*np.cos(np.deg2rad(offset_start + i*(180/n)))])

a = 0.3764210592453014, b = 1.068500263848051
